[TSFresh for Time Series Feature Extraction in Python](https://medium.com/@kylejones_47003/tsfresh-for-time-series-feature-extraction-in-python-c4ee791f467c?source=your_stories_page-------------------------------------)
===========================================================================================================================================================================================================================

by: Kyle Jones\
published: 2025-01-10

In [ ]:
! pip install tsfresh

In [3]:
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction.feature_calculators import mean
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create a larger sample dataset
np.random.seed(42)
n_series = 100
n_timepoints = 100

time_series_list = []
for i in range(n_series):
    frequency = np.random.uniform(0.5, 2)
    phase = np.random.uniform(0, 2*np.pi)
    noise_level = np.random.uniform(0.05, 0.2)
    
    values = np.sin(frequency * np.linspace(0, 10, n_timepoints) + phase) + np.random.normal(0, noise_level, n_timepoints)
    
    df = pd.DataFrame({
        'id': i,
        'time': range(n_timepoints),
        'value': values
    })
    time_series_list.append(df)

time_series = pd.concat(time_series_list, ignore_index=True)

print("Original time series data:")
print(time_series.head())
print(f"Number of time series: {n_series}")
print(f"Number of timepoints per series: {n_timepoints}")

# Visualize some of the time series
plt.figure(figsize=(12, 6))
for i in range(5):  # Plot first 5 time series
    plt.plot(time_series[time_series['id'] == i]['time'], 
             time_series[time_series['id'] == i]['value'], 
             label=f'Series {i}')
plt.title('Sample of Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.savefig("sample_TS.png")
plt.show()

# Extract features
features = extract_features(time_series, column_id="id", column_sort="time", n_jobs=0)

print("\nExtracted features:")
print(features.head())

# Impute NaN values
features_imputed = impute(features)

# Create a target variable (classify based on frequency)
target = pd.Series(index=range(n_series), dtype=int)
target[features_imputed.index % 2 == 0] = 0  # Even indices
target[features_imputed.index % 2 == 1] = 1  # Odd indices

# Select relevant features
selected_features = select_features(features_imputed, target)

# If no features were selected, use all features
if selected_features.empty:
    print("\nNo features were selected. Using all features.")
    selected_features = features_imputed
else:
    print("\nSelected features:")
    print(selected_features.head())

print(f"\nNumber of features: {selected_features.shape[1]}")

print("\nNames of features (first 10):")
print(selected_features.columns.tolist()[:10])

# Classification part
# Split data for classification
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    selected_features, target, test_size=0.2, random_state=42
)

# Train a classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_clf, y_train_clf)

# Evaluate the classification model
y_pred_clf = clf.predict(X_test_clf)
print("\nClassification Model Performance:")
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf):.2f}")
print("\nClassification Report:")
print(classification_report(y_test_clf, y_pred_clf))

# Visualize confusion matrix
cm = confusion_matrix(y_test_clf, y_pred_clf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig("confusion_matrix.png")
plt.show()

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train_clf.columns,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# Visualize feature importance
plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Most Important Features')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.savefig("feature_importance.png")
plt.show()


ImportError: Numba needs NumPy 1.20 or less

In [ ]:
# Multivariate feature extraction
# Add another variable to the dataset
time_series["value2"] = time_series["value"] * 0.5 + np.random.normal(0, 0.05, len(time_series))

# Extract features from both variables
features_multivariate = extract_features(
    time_series, 
    column_id="id", 
    column_sort="time", 
    default_fc_parameters=EfficientFCParameters(),
    n_jobs=0
)
print("\nMultivariate features:")
print(features_multivariate.head())

In [ ]:
# Compute the mean for each time series
custom_features = time_series.groupby("id")["value"].apply(mean)
print("\nCustom features (mean of each time series, first 5):")
print(custom_features.head())

# Visualize the distribution of custom features
plt.figure(figsize=(10, 6))
sns.histplot(custom_features, kde=True)
plt.title('Distribution of Mean Values for Each Time Series')
plt.xlabel('Mean Value')
plt.ylabel('Count')
plt.savefig("dist_of_means_TS.png")
plt.show()

# Visualize the relationship between custom features and target
plt.figure(figsize=(10, 6))
sns.scatterplot(x=custom_features, y=target)
plt.title('Relationship between Mean Values and Target')
plt.xlabel('Mean Value')
plt.ylabel('Target')
plt.savefig("means_v_target_TS.png")
plt.show()
